# The battle of Neighborhoods - Stockholm

This project aims to find out more about different places to live in Stockholm and for new people to Stockholm to find the best Neighborhood for them.

Let start with downloading all the dependencies we will need for the problem

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

import geopy
from geopy.geocoders import Nominatim

import sys

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.3                |   py37hc8dfbb8_1         3.0 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    python_abi-3.7             |          1_cp37m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50

Load data with postalcodes from neighborhoods with apartments in Stockholm

In [25]:
# Read the csv data downloaded from hemnet with available apartments in Stockholm
data = pd.read_csv('addresser.csv', sep=';')

# View the first ten rows
data.head(10)

,adress
0,Sandelsgatan 30
1,Birger Jarlsgatan 113c
2,Pollargatan 24
3,Nybohovsbacken 83
4,Parkgatan 14
5,Kalmgatan 44
6,Storgatan 59
7,John Bergs Plan 3
8,Essinge Brogata 3
9,Lindhagensgatan 149


From mapbox load latitude and longitude for the postal codes labeled "postnummer" and add to dataframe

In [26]:
data.shape
data.columns = ['adress']
data.head()
data.columns

Index(['adress'], dtype='object')

In [27]:
geodata = pd.DataFrame(columns=['adress', 'city', 'latitude', 'longitude'])
geodata.head()

,adress,city,latitude,longitude


In [28]:
#quick check if the function works in Sweden
geolocator = Nominatim(user_agent="myGeocoder")
location = geolocator.geocode("sagagatan 12 stockholm")

print((location.latitude, location.longitude))
print('Geolocator works in Sweden!')

(59.3569498, 18.001266)
Geolocator works in Sweden!


In [29]:
#Loop through data file, and fine lat and long for every adress in Stockholm
#If lat and long can be found add to geodata dataframe

for i in range(len(data)):
    try:
        location = geolocator.geocode(data.loc[i].at['adress']+' Stockholm')
        geodata.loc[i] = (data.loc[i].at['adress'],' Stockholm', location.latitude, location.longitude)
    except:
        pass

In [30]:
#See if everything looks like we want it
geodata.head()

,adress,city,latitude,longitude
0,Sandelsgatan 30,Stockholm,59.344188,18.091676
1,Birger Jarlsgatan 113c,Stockholm,59.351005,18.056764
2,Pollargatan 24,Stockholm,59.310423,18.102696
3,Nybohovsbacken 83,Stockholm,59.307940,18.013844
4,Parkgatan 14,Stockholm,59.333393,18.035737


After creating my geodata dataframe with all available apartments in Stockholms adresses, I will double check my results by vizualize the adresses on a map

In [31]:
# create map of Stockholm using latitude and longitude values
latitude = 59.334591
longitude = 18.063240
map_stockholm = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(geodata['latitude'], geodata['longitude'], geodata['adress']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_stockholm)  
    
map_stockholm

After making a dataframe with adresses with available apartments and their latitude and longitude, I'm gonna use foursquares API to explore the neighborhoods and segment them

Define Foursquare Credentials and Version

In [34]:
CLIENT_ID = '4ZWZG1FJ2OVWSKWPZXTWCJDCJSNEMI3OOTRE5ONHU2KT4LHJ' # your Foursquare ID
CLIENT_SECRET = '1NH3SNUGPX151UKBBFVT0J1X0CPX32DO4JDHGKDNDZ3HJ1E2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4ZWZG1FJ2OVWSKWPZXTWCJDCJSNEMI3OOTRE5ONHU2KT4LHJ
CLIENT_SECRET:1NH3SNUGPX151UKBBFVT0J1X0CPX32DO4JDHGKDNDZ3HJ1E2


Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [43]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [41]:
stockholm_venues = getNearbyVenues(names=geodata['adress'],
                                   latitudes=geodata['latitude'],
                                   longitudes=geodata['longitude']
                                  )


Sandelsgatan 30
Birger Jarlsgatan 113c
Pollargatan 24
Nybohovsbacken 83
Parkgatan 14
Kalmgatan 44
Storgatan 59
John Bergs Plan 3
Essinge Brogata 3
Lindhagensgatan 149
Surbrunnsgatan 7 
Birkagatan 14
Primusgatan 58
Hornsgatan 150 B
Sandelsgatan 38
Kaplansbacken 6
Danderydsgatan 30
Sankt Eriksplan 6A
Birger Jarlsgatan 97
Upplandsgatan 67
Grev Turegatan 54
Styrmansgatan 25
Roslagsgatan 15A
Primusgatan 78
Bondegatan 27
Frejgatan 19
Heleneborgsgatan 40
Heimdalsgatan 6
Lundagatan 36f
Roslagsgatan 37
Torsgatan 70
Upplandsgatan 49
Styrmansgatan 21
Strindbergsgatan 53
Skeppargatan 68
Studentbacken 32
Birger Jarlsgatan 123
Luxgatan 23
Stallgatan 26
Roslagsgatan 15
Sandelsgatan 42
Tomtebogatan 30
Scheelegatan 6 A 
Brahegatan 62
Atlasgatan 4
Roslagsgatan 50A
Korphoppsgatan 20
Sankt Eriksgatan 67
Wittstocksgatan 3 c
Svarvargatan 7
Observatoriegatan 21
Jungfrugatan 64
Grubbens Gata 9
Tomtebogatan 29
Sandhamnsgatan 75
Vanadisvagen 13
Sankt eriksplan 11
sveavagen 80
sveavagen 2
odengatan 14
kungsgatan

In [52]:
stockholm_venues.head()


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Sandelsgatan 30,59.344188,18.091676,Mä-Took,59.342401,18.089964,Thai Restaurant
1,Sandelsgatan 30,59.344188,18.091676,Delantina,59.344119,18.092917,Café
2,Sandelsgatan 30,59.344188,18.091676,Diwan,59.340733,18.091595,Persian Restaurant
3,Sandelsgatan 30,59.344188,18.091676,Ett bageri,59.343634,18.086632,Bakery
4,Sandelsgatan 30,59.344188,18.091676,Paraden Kvarterskrog,59.341053,18.090264,Modern European Restaurant


In [53]:
stockholm_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Atlasgatan 4,43,43,43,43,43,43
Birger Jarlsgatan 113c,13,13,13,13,13,13
Birger Jarlsgatan 123,23,23,23,23,23,23
Birger Jarlsgatan 97,25,25,25,25,25,25
Birkagatan 14,44,44,44,44,44,44
Bondegatan 27,100,100,100,100,100,100
Brahegatan 62,18,18,18,18,18,18
Danderydsgatan 30,25,25,25,25,25,25
Essinge Brogata 3,21,21,21,21,21,21


Analyze each Neighborhood

In [57]:
stockholm_onehot = pd.get_dummies(stockholm_venues[['Venue Category']], prefix="", prefix_sep="")
stockholm_onehot.insert(loc=0, column='Neighborhood', value=stockholm_venues['Neighborhood'] )
stockholm_onehot.shape

(5217, 224)

In [58]:
stockholm_grouped = stockholm_onehot.groupby('Neighborhood').mean().reset_index()
stockholm_grouped.head()

,Neighborhood,Accessories Store,Advertising Agency,American Restaurant,Amphitheater,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Building,Burger Joint,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Camera Store,Canal,Candy Store,Caribbean Restaurant,Castle,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cupcake Shop,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Dive Bar,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General College & University,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hockey Arena,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Mongolian Restaurant,Monument / Landmark,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Newsstand,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Gym,Outdoor Sculpture,Outdoor Supply Store,Outlet Store,Palace,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pharmacy,Photography Lab,Photography Studio,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pop-Up Shop,Pub,Ramen Restaurant,Record Shop,Restaurant,Road,Roof Deck,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Squash Court,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Taverna,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tiki Bar,Tourist Information Center,Track,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Atlasgatan 4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023256,0.000000,0.0,0.0,0.0,0.0,0.0,0.069767,0.0,0.0,0.0,0.023256,0.0,0.0,0.0,0.023256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.046512,0.069767,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023256,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.023256,0.0,0.0,0.0,0.0,0.0,0.046512,0.000000,0.000000,0.0,0.0,0.0,0.069767,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.023256,0.000000,0.023256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.093023,0.023256,0.000000,0.0,0.0,0.0,

In [59]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [61]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = stockholm_grouped['Neighborhood']

for ind in np.arange(stockholm_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(stockholm_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Atlasgatan 4,Pizza Place,Ice Cream Shop,Coffee Shop,Bakery,Thai Restaurant,Hostel,Restaurant,Cocktail Bar,Sushi Restaurant,Pub
1,Birger Jarlsgatan 113c,Sushi Restaurant,Squash Court,Playground,Scandinavian Restaurant,Miscellaneous Shop,Italian Restaurant,Plaza,Electronics Store,Pizza Place,Sporting Goods Shop
2,Birger Jarlsgatan 123,Scandinavian Restaurant,Hotel,Sushi Restaurant,Burger Joint,Pizza Place,Playground,Plaza,Electronics Store,Café,Miscellaneous Shop
3,Birger Jarlsgatan 97,Scandinavian Restaurant,Italian Restaurant,Café,Park,Pizza Place,Salad Place,Mediterranean Restaurant,French Restaurant,Spanish Restaurant,Sporting Goods Shop
4,Birkagatan 14,Ice Cream Shop,Pizza Place,Indian Restaurant,Sushi Restaurant,French Restaurant,Cocktail Bar,Coffee Shop,Café,Bakery,Thai Restaurant


In [102]:
# set number of clusters
kclusters = 7

stockholm_grouped_clustering = stockholm_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(stockholm_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:7]

array([2, 0, 0, 0, 2, 1, 2], dtype=int32)

In [103]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

stockholm_merged = geodata

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
stockholm_merged = stockholm_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='adress')

stockholm_merged.head()

,adress,city,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Sandelsgatan 30,Stockholm,59.344188,18.091676,6,Bus Stop,Thai Restaurant,Café,Restaurant,Bakery,Convenience Store,Pizza Place,Burger Joint,Chocolate Shop,Clothing Store
1,Birger Jarlsgatan 113c,Stockholm,59.351005,18.056764,0,Sushi Restaurant,Squash Court,Playground,Scandinavian Restaurant,Miscellaneous Shop,Italian Restaurant,Plaza,Electronics Store,Pizza Place,Sporting Goods Shop
2,Pollargatan 24,Stockholm,59.310423,18.102696,5,Bus Stop,Park,Pizza Place,Italian Restaurant,Beer Garden,Grocery Store,Asian Restaurant,Harbor / Marina,Café,Pier
3,Nybohovsbacken 83,Stockholm,59.307940,18.013844,3,Stadium,Department Store,Café,Athletics & Sports,Electronics Store,Yoga Studio,Falafel Restaurant,Fountain,Food Truck,Food Court
4,Parkgatan 14,Stockholm,59.333393,18.035737,2,Indian Restaurant,Thai Restaurant,Bakery,Sushi Restaurant,Gym / Fitness Center,Steakhouse,Burger Joint,Middle Eastern Restaurant,Park,Italian Restaurant


In [104]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,Atlasgatan 4,Pizza Place,Ice Cream Shop,Coffee Shop,Bakery,Thai Restaurant,Hostel,Restaurant,Cocktail Bar,Sushi Restaurant,Pub
1,0,Birger Jarlsgatan 113c,Sushi Restaurant,Squash Court,Playground,Scandinavian Restaurant,Miscellaneous Shop,Italian Restaurant,Plaza,Electronics Store,Pizza Place,Sporting Goods Shop
2,0,Birger Jarlsgatan 123,Scandinavian Restaurant,Hotel,Sushi Restaurant,Burger Joint,Pizza Place,Playground,Plaza,Electronics Store,Café,Miscellaneous Shop
3,0,Birger Jarlsgatan 97,Scandinavian Restaurant,Italian Restaurant,Café,Park,Pizza Place,Salad Place,Mediterranean Restaurant,French Restaurant,Spanish Restaurant,Sporting Goods Shop
4,2,Birkagatan 14,Ice Cream Shop,Pizza Place,Indian Restaurant,Sushi Restaurant,French Restaurant,Cocktail Bar,Coffee Shop,Café,Bakery,Thai Restaurant


In [105]:
address = 'Stockholm'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Stockholm are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Stockholm are 59.3251172, 18.0710935.


Creating a map

In [106]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(stockholm_merged['latitude'], stockholm_merged['longitude'], stockholm_merged['adress'], stockholm_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine Clusters

# Cluster 1

In [118]:
stockholm_merged.loc[stockholm_merged['Cluster Labels'] == 0, stockholm_merged.columns[[0] + list(range(5, stockholm_merged.shape[1]))]]

,adress,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Birger Jarlsgatan 113c,Sushi Restaurant,Squash Court,Playground,Scandinavian Restaurant,Miscellaneous Shop,Italian Restaurant,Plaza,Electronics Store,Pizza Place,Sporting Goods Shop
10,Surbrunnsgatan 7,Scandinavian Restaurant,Café,Italian Restaurant,Sushi Restaurant,Gym / Fitness Center,Coffee Shop,Bakery,Salad Place,Mediterranean Restaurant,Spanish Restaurant
18,Birger Jarlsgatan 97,Scandinavian Restaurant,Italian Restaurant,Café,Park,Pizza Place,Salad Place,Mediterranean Restaurant,French Restaurant,Spanish Restaurant,Sporting Goods Shop
23,Roslagsgatan 15A,Scandinavian Restaurant,Italian Restaurant,Middle Eastern Restaurant,Burger Joint,Vegetarian / Vegan Restaurant,Café,French Restaurant,Pizza Place,Salad Place,Breakfast Spot
28,Frejgatan 19,Scandinavian Restaurant,Italian Restaurant,Mediterranean Restaurant,Chinese Restaurant,Park,French Restaurant,Spanish Restaurant,Sporting Goods Shop,Café,Salad Place
34,Roslagsgatan 37,Scandinavian Restaurant,Pizza Place,Park,Theater,Miscellaneous Shop,Café,Spanish Restaurant,Sporting Goods Shop,Supermarket,Sushi Restaurant
45,Birger Jarlsgatan 123,Scandinavian Restaurant,Hotel,Sushi Restaurant,Burger Joint,Pizza Place,Playground,Plaza,Electronics Store,Café,Miscellaneous Shop
50,Roslagsgatan 15,Scandinavian Restaurant,Italian Restaurant,French Restaurant,Burger Joint,Thrift / Vintage Store,Spanish Restaurant,Sushi Restaurant,Chinese Restaurant,General College & University,Tea Room
59,Roslagsgatan 50A,Scandinavian Restaurant,Italian Restaurant,Middle Eastern Restaurant,Burger Joint,Vegetarian / Vegan Restaurant,Café,French Restaurant,Pizza Place,Salad Place,Breakfast Spot
80,odengatan 14,Café,Scandinavian Restaurant,Gym / Fitness Center,Italian Restaurant,Sushi Restaurant,Chinese Restaurant,Spanish Restaurant,Burger Joint,Bus Station,Salad Place


# Cluster 2

In [120]:
stockholm_merged.loc[stockholm_merged['Cluster Labels'] == 1, stockholm_merged.columns[[0] + list(range(5, stockholm_merged.shape[1]))]]

,adress,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,John Bergs Plan 3,Hotel,Café,Gym / Fitness Center,Metro Station,Bakery,Plaza,Office,Castle,Scandinavian Restaurant,Scenic Lookout
26,Bondegatan 27,Café,Clothing Store,Scandinavian Restaurant,Thai Restaurant,Coffee Shop,Indian Restaurant,Record Shop,Italian Restaurant,Deli / Bodega,Beer Bar
61,Korphoppsgatan 20,Café,Gym / Fitness Center,Scandinavian Restaurant,Hotel,Bakery,Sushi Restaurant,Indian Restaurant,Candy Store,Furniture / Home Store,Light Rail Station
78,sveavagen 2,Hotel,Scandinavian Restaurant,Clothing Store,Coffee Shop,Café,Gym / Fitness Center,Asian Restaurant,Cocktail Bar,Burger Joint,Department Store
81,kungsgatan 12,Scandinavian Restaurant,Hotel,Italian Restaurant,Movie Theater,Asian Restaurant,Burger Joint,Coffee Shop,Clothing Store,Café,Bakery
88,kungsgatan 9,Scandinavian Restaurant,Hotel,Movie Theater,Italian Restaurant,Gym / Fitness Center,Coffee Shop,Burger Joint,Café,Clothing Store,Asian Restaurant
89,klarabergsviadukten 49,Hotel,Café,Coffee Shop,Hotel Bar,Scandinavian Restaurant,Plaza,Asian Restaurant,Indian Restaurant,Sandwich Place,Italian Restaurant
90,master samuelsgatan 71,Hotel,Café,Gym / Fitness Center,Asian Restaurant,Burger Joint,Boat or Ferry,Camera Store,Plaza,Clothing Store,Cocktail Bar
94,olof palmes gata 15,Café,Hotel,Gym / Fitness Center,Asian Restaurant,Pub,Scandinavian Restaurant,Indian Restaurant,Middle Eastern Restaurant,Italian Restaurant,Juice Bar
100,klarabergsgatan 30,Hotel,Café,Coffee Shop,Bakery,Scandinavian Restaurant,Clothing Store,Asian Restaurant,Cocktail Bar,Gym / Fitness Center,Burger Joint


# Cluster 4

In [121]:
stockholm_merged.loc[stockholm_merged['Cluster Labels'] == 3, stockholm_merged.columns[[0] + list(range(5, stockholm_merged.shape[1]))]]

,adress,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Nybohovsbacken 83,Stadium,Department Store,Café,Athletics & Sports,Electronics Store,Yoga Studio,Falafel Restaurant,Fountain,Food Truck,Food Court


# Cluster 5

In [122]:
stockholm_merged.loc[stockholm_merged['Cluster Labels'] == 4, stockholm_merged.columns[[0] + list(range(5, stockholm_merged.shape[1]))]]

,adress,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Storgatan 59,Scandinavian Restaurant,Restaurant,Park,Bus Station,Museum,Tram Station,Café,Fast Food Restaurant,Liquor Store,Furniture / Home Store
16,Danderydsgatan 30,Scandinavian Restaurant,Café,Sushi Restaurant,Light Rail Station,Italian Restaurant,Chinese Restaurant,Greek Restaurant,Stadium,Medical Center,General College & University
20,Grev Turegatan 54,Scandinavian Restaurant,Italian Restaurant,Deli / Bodega,Bar,Bakery,Park,Grocery Store,Pizza Place,French Restaurant,Yoga Studio
21,Styrmansgatan 25,Scandinavian Restaurant,Italian Restaurant,Grocery Store,Café,Sushi Restaurant,Yoga Studio,Thai Restaurant,French Restaurant,History Museum,Restaurant
39,Styrmansgatan 21,Scandinavian Restaurant,Italian Restaurant,Sushi Restaurant,Grocery Store,Café,Thai Restaurant,History Museum,French Restaurant,Yoga Studio,Tram Station
48,Stallgatan 26,Scandinavian Restaurant,Hotel,Clothing Store,Café,Theater,Liquor Store,Restaurant,Bistro,Cheese Shop,Museum
83,hamngatan 20,Scandinavian Restaurant,Hotel,Gym / Fitness Center,Clothing Store,Coffee Shop,Plaza,Cocktail Bar,Asian Restaurant,Café,Burger Joint
95,norrlandsgatan 7,Scandinavian Restaurant,Hotel,Gym / Fitness Center,Coffee Shop,Clothing Store,Italian Restaurant,Burger Joint,Asian Restaurant,Restaurant,Food Court
97,norrlandsgatan 38,Scandinavian Restaurant,Hotel,Gym / Fitness Center,Coffee Shop,Clothing Store,Burger Joint,Asian Restaurant,Seafood Restaurant,Italian Restaurant,Historic Site
117,stora nygatan 19,Scandinavian Restaurant,History Museum,Wine Bar,Plaza,Bakery,Historic Site,Gastropub,Coffee Shop,Cocktail Bar,Pub


# Cluster 6

In [123]:
stockholm_merged.loc[stockholm_merged['Cluster Labels'] == 5, stockholm_merged.columns[[0] + list(range(5, stockholm_merged.shape[1]))]]

,adress,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Pollargatan 24,Bus Stop,Park,Pizza Place,Italian Restaurant,Beer Garden,Grocery Store,Asian Restaurant,Harbor / Marina,Café,Pier
8,Essinge Brogata 3,Grocery Store,Bus Stop,Bakery,Scandinavian Restaurant,Restaurant,Gym / Fitness Center,Salad Place,Beach,Thai Restaurant,Indian Restaurant
12,Primusgatan 58,Bus Stop,Pizza Place,Thai Restaurant,Harbor / Marina,Grocery Store,Restaurant,Park,Pier,Beach,Café
24,Primusgatan 78,Thai Restaurant,Park,Pizza Place,Harbor / Marina,Restaurant,Grocery Store,Beach,Bus Stop,Pier,Deli / Bodega
47,Luxgatan 23,Thai Restaurant,Park,Pizza Place,Harbor / Marina,Restaurant,Grocery Store,Bus Stop,Pier,Deli / Bodega,Café
114,tegelviksgatan 68,Café,Park,Italian Restaurant,Brewery,Mediterranean Restaurant,Restaurant,Sushi Restaurant,Theater,Grocery Store,Gym / Fitness Center
118,tegelviksgatan 43,Café,Pizza Place,Park,Asian Restaurant,Pier,Bus Stop,Beer Garden,Ice Cream Shop,Amphitheater,Fried Chicken Joint


# Cluster 7

In [124]:
stockholm_merged.loc[stockholm_merged['Cluster Labels'] == 6, stockholm_merged.columns[[0] + list(range(5, stockholm_merged.shape[1]))]]

,adress,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Sandelsgatan 30,Bus Stop,Thai Restaurant,Café,Restaurant,Bakery,Convenience Store,Pizza Place,Burger Joint,Chocolate Shop,Clothing Store
14,Sandelsgatan 38,Thai Restaurant,Sushi Restaurant,Café,Convenience Store,Burger Joint,Bus Stop,Restaurant,Chocolate Shop,Clothing Store,Coffee Shop
41,Strindbergsgatan 53,Café,Coffee Shop,Park,Thai Restaurant,Bakery,Convenience Store,Clothing Store,Persian Restaurant,Bistro,Supermarket
44,Studentbacken 32,Bus Stop,Asian Restaurant,Sushi Restaurant,Thai Restaurant,Fast Food Restaurant,Burger Joint,Tennis Stadium,Pizza Place,Scandinavian Restaurant,Grocery Store
51,Sandelsgatan 42,Thai Restaurant,Park,Café,Sushi Restaurant,Persian Restaurant,Pizza Place,Coffee Shop,Clothing Store,Electronics Store,Chocolate Shop
68,Jungfrugatan 64,Bakery,Café,Pizza Place,Thai Restaurant,Coffee Shop,Electronics Store,Sushi Restaurant,Supermarket,Stadium,Chocolate Shop
